In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('/data/training_data.csv')

In [ ]:
df.head()

,ID,timestamp,cpu_type,cpu_limit,ram_limit,cpu_usage,ram_usage,num_req,conc_lvl,latency
0,2714,2021-11-04 23:15:38,Intel Xeon Gold 5317,2.0,70M,0.084450,46854144.0,100,71,7429000
1,38113,2021-11-04 23:23:27,AMD EPYC 72F3,2.0,70M,0.277893,55934976.0,300,21,2196000
2,33409,2021-11-11 13:32:59,Intel Xeon Platinum 8380,2.5,66M,0.351042,59109376.0,700,1,37000
3,10123,2022-03-19 02:35:05,Intel Xeon Platinum 8356H,0.8,768M,0.026230,43659264.0,100,41,3946000
4,12815,2022-03-17 06:30:29,AMD EPYC 72F3,0.4,512M,0.108346,51048448.0,800,51,5084000


In [ ]:
df.columns

Index(['ID', 'timestamp', 'cpu_type', 'cpu_limit', 'ram_limit', 'cpu_usage',
       'ram_usage', 'num_req', 'conc_lvl', 'latency'],
      dtype='object')

In [ ]:
df.shape

(41134, 10)

In [ ]:
len(df['cpu_type'].unique())

107

In [ ]:
df.drop('ID', axis = 1, inplace = True)

In [ ]:
df['cpu_type'] = df['cpu_type'].map(df['cpu_type'].value_counts()) / len(df)

In [ ]:
df['ram_limit'].unique()

array(['70M', '66M', '768M', '512M', '640M', '100M', '95M', '256M', '75M',
       '384M', '128M', '896M', '69M', '72M', '71M', '500M', '73M', '80M',
       '68M', '67M', '200M', '74M', '1024M', '65M', '400M', '85M', '450M',
       '90M', '150M', '250M', '300M', '77M', '63M', '64M', '79M', '76M',
       '350M', '78M'], dtype=object)

In [ ]:
df['ram_limit'] = df['ram_limit'].str.replace('M', '').astype(int)

In [ ]:
df.dtypes

timestamp     object
cpu_type     float64
cpu_limit    float64
ram_limit      int64
cpu_usage    float64
ram_usage    float64
num_req        int64
conc_lvl       int64
latency        int64
dtype: object

In [ ]:
df['timestamp'].unique()

array(['2021-11-04 23:15:38', '2021-11-04 23:23:27',
       '2021-11-11 13:32:59', ..., '2022-03-18 02:05:09',
       '2022-03-16 16:28:38', '2022-03-17 03:18:29'], dtype=object)

In [ ]:
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Extract date and time components
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek

# df['week_of_year'] = df['timestamp'].dt.isocalendar().week

# Drop the original timestamp column
df = df.drop(['timestamp'], axis=1)

In [ ]:
X = df.drop('latency', axis=1)
y = df['latency']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train = X_train.drop('day_of_week', axis = 1)
X_test = X_test.drop('day_of_week', axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize features
scaler = StandardScaler()
X_stack = scaler.fit_transform(X)
# X_test_stack = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Model training using Random Forest
model = RandomForestRegressor(n_estimators=30,random_state=42)
model.fit(X_stack, y)

# Model evaluation
# y_pred = model.predict(X_test)
# print(f'R² score: {r2_score(y_test, y_pred)}')

RandomForestRegressor(n_estimators=30, random_state=42)

In [ ]:
# train using randomforest with gridSearch for the best parameters

import numpy as np
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define the grid search object
grid_search = GridSearchCV(model, param_grid, cv=5)

# Fit the grid search object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
model3 = RandomForestRegressor(**best_params)
model3.fit(X_train, y_train)

# Make predictions
y_pred = model3.predict(X_test)

# Calculate the R^2 score
r2_score = r2_score(y_test, y_pred)

# Print the score and RMSE
print("Score:", r2_score)


Score: 0.97282336218705
